We are Implementing the Grouding DINO + SAM creating a dataset stroing it and testing on the yolo v8. Run this file using the Colab which would be easy to create and use the datasets and models.

* upload the videos.zip that is in the zip file to the videos directory that is created as part of this code.   




In [ ]:
#code for implementing Grounded DINO+SAM model
#install necessary Lib's
!pip install -q \autodistill \autodistill-grounded-sam \autodistill-yolov8 \ supervision==0.9.0
!pip install roboflow

In [1]:
import os
HOME = os.getcwd()
!mkdir {HOME}/images
!mkdir {HOME}/videos

 # adding the zipped videos that we want to implement and pre process

In [ ]:
%cd {HOME}/videos
!unzip vid_zip.zip # give videos zip file name

In [3]:
Videos_dir_path =  f"{HOME}/videos/videos.zip" # give Videos path to be processed
Image_dir_path =  f"{HOME}/images" # give path of the images to be stored
Noframes = 70 # setting the frame stride

In [4]:
import supervision as sv
from tqdm.notebook import tqdm

# extracting the videos from the path if it is a mp4.
video_paths = sv.list_files_with_extensions(directory=Videos_dir_path,extensions=["mov","mp4"])

#extracting the frames from the video
for video_path in tqdm(video_paths):
    video_name = video_path.stem
    image_name_pattern = video_name + "-{:05d}.png"
    with sv.ImageSink(target_dir_path=Image_dir_path, image_name_pattern=image_name_pattern) as sink:
        for image in sv.get_video_frames_generator(source_path=str(video_path), stride=Noframes):
            sink.save_image(image=image)

  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
# Adding the CaptionOntology which is the text part that we send to the the Grounding DINO
from autodistill.detection.caption_ontology import CaptionOntology
ontology=CaptionOntology({
  "person": "person",
 "bicycle": "bicycle",
 "car": "car",
 "motorcycle": "motorcycle",
 "airplane": "airplane",
 "bus": "bus",
 "train": "train",
 "truck": "truck",
 "boat": "boat",
 "traffic light": "traffic light",
 "fire hydrant": "fire hydrant",
 "stop sign": "stop sign",
 "parking meter": "parking meter",
 "bench": "bench",
 "bird": "bird",
 "Animal":"Animal",
 "backpack": "backpack",
 "umbrella": "umbrella",
 "handbag": "handbag",
 "tie": "tie",
 "suitcase": "suitcase",
 "frisbee": "frisbee",
 "skis": "skis",
 "snowboard": "snowboard",
 "sports ball": "sports ball",
 "kite": "kite",
 "baseball bat": "baseball bat",
 "baseball glove": "baseball glove",
 "skateboard": "skateboard",
 "surfboard": "surfboard",
 "tennis racket": "tennis racket",
 "bottle": "bottle",
 "wine glass": "wine glass",
 "cup": "cup",
 "fork": "fork",
 "knife": "knife",
 "spoon": "spoon",
 "bowl": "bowl",
 "banana": "banana",
 "apple": "apple",
 "sandwich": "sandwich",
 "orange": "orange",
 "broccoli": "broccoli",
 "carrot": "carrot",
 "hot dog": "hot dog",
 "pizza": "pizza",
 "donut": "donut",
 "cake": "cake",
 "chair": "chair",
 "couch": "couch",
 "potted plant": "potted plant",
 "bed": "bed",
 "dining table": "dining table",
 "toilet": "toilet",
 "tv": "tv",
 "laptop": "laptop",
 "mouse": "mouse",
 "remote": "remote",
 "keyboard": "keyboard",
 "cell phone": "cell phone",
 "Telephone" : "Telephone",
 "oven": "oven",
 "toaster": "toaster",
 "sink": "sink",
 "refrigerator": "refrigerator",
 "book": "book",
 "clock": "clock",
 "vase": "vase",
 "scissors": "scissors",
 "teddy bear": "teddy bear",
 "hair drier": "hair drier",
 "toothbrush": "toothbrush",
 "building": "building",
 "Jewelry":"Jewelry",
  "Ring" : "Ring"
})

In [6]:
DATASET_DIR_PATH = f"{HOME}/dataset" # setting path to store the dataset created from the Grounded DINO + SAM Model

In [ ]:
from autodistill_grounded_sam import GroundedSAM
# implement the Grounded SAM from autodistill frame work of roboflow.
# this is a combination of Grounded dino + Segement anything Model.
base_model = GroundedSAM(ontology=ontology)
dataset = base_model.label(
    input_folder=Image_dir_path,
    extension=".png",
    output_folder=DATASET_DIR_PATH)
# it creates a dataset similar to COCO and stores it in given dir_path

In [19]:
import matplotlib.pyplot as plt
import random
SAMPLE_SIZE = 18
image_names = list(dataset.images.keys())[:SAMPLE_SIZE]

mask_annotator = sv.MaskAnnotator()
box_annotator = sv.BoxAnnotator()

for image_name in image_names:
    image = dataset.images[image_name]
    annotations = dataset.annotations[image_name]
    labels = [
        dataset.classes[class_id]
        for class_id in annotations.class_id]

    annotated_image = mask_annotator.annotate(
        scene=image.copy(),
        detections=annotations)
    annotated_image = box_annotator.annotate(
        scene=annotated_image,
        detections=annotations,
        labels=labels)

    # Generate random plot sizes between 4 inches and 10 inches
    random_width = random.uniform(4, 10)
    random_height = random.uniform(4, 10)

    # Plot each image individually with a random size
    plt.figure(figsize=(random_width, random_height))
    plt.imshow(annotated_image)
    plt.title(image_name)
    plt.axis('off')  # Hide axes
    plt.show()

In [8]:
# setting up Paths to the content created from the DINO + SAM Model
ANNOTATIONS_DIRECTORY_PATH = f"{HOME}/dataset/train/labels"
IMAGES_DIRECTORY_PATH = f"{HOME}/dataset/train/images"
DATA_YAML_PATH = f"{HOME}/dataset/data.yaml"

In [ ]:
from autodistill_yolov8 import YOLOv8
# training the yolov8 model on the dataset that we have generated from the Grouding DINO + SAM
target_model = YOLOv8("yolov8l.pt")
target_model.train(DATA_YAML_PATH, epochs=10)

In [ ]:
# Test the model on an image
target_model.predict("/content/images/k3569b16n-30000157759113_013_mezzCrop-00018.png")


In [ ]:
from IPython.display import Image
Image(filename=f'{HOME}/runs/detect/train/val_batch0_pred.jpg', width=600)

In [ ]:
TRAINED_MODEL_PATH = f"{HOME}/runs/detect/train/weights/best.pt"
!yolo predict model={TRAINED_MODEL_PATH} source={"/content/images/k3569b16n-30000157759113_013_mezzCrop-00018.png"}"